In [259]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings("ignore")


In [260]:
from zipfile import ZipFile
file_name = "data/mapping-mobilities-csv.zip"
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

In [261]:
passaporti_csv = "form-1__passports.csv"
rinnovi_csv = "branch-1__rinnovi-inserire-un-branch-per-ogni-rinnovo-presente.csv"


In [262]:
passaporti = pd.read_csv(passaporti_csv, sep=",")

In [263]:
del passaporti["ec5_uuid"]
del passaporti["created_at"]
del passaporti["uploaded_at"]
del passaporti["title"]

In [264]:
passaporti.drop_duplicates(inplace=True)

In [265]:
passaporti["nome_raw"] = passaporti["nome"]

In [266]:
passaporti.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

In [267]:
colonne = list(passaporti.columns)

In [268]:
for colonna in colonne:
    try:
        passaporti[colonna] = passaporti[colonna].str.lstrip()
        passaporti[colonna] = passaporti[colonna].str.rstrip()
        passaporti[colonna] = passaporti[colonna].str.replace(")","")
        passaporti[colonna] = passaporti[colonna].str.replace("(", "")
    except Exception as e:
        if e == "Can only use .str accessor with string values!": 
            pass

In [269]:
colonne = ['nome_raw',"luogo_nascita",'espatri']

In [270]:
for colonna in colonne:
    try:
        passaporti[colonna] = passaporti[colonna].str.title()
    except Exception as e:
        if e == "Can only use .str accessor with string values!":
            pass

In [271]:
def getsplitfirst(value):
    value = value.split(" ")
    return value[0]

In [272]:
def getaftersplit(value):
    nome = value.split(" ")
    resto = value.replace(nome[0],"")
    return resto


In [273]:
passaporti['cognome'] = passaporti['nome_raw'].apply(lambda x:getsplitfirst(x))

In [274]:
passaporti['nome'] = passaporti['nome_raw'].apply(lambda x:getaftersplit(x))

In [275]:
passaporti['nome'] = passaporti['nome'].str.lstrip()

In [276]:
cognomi_doppi = [
    "Eberle", "Oss", "Bals", "De", "Fatis",
    "Mazzurana", "Ferrari", "Ehmayer",
    "Pinter", "Papot", "Negri", "Giovanelli"
    "Porta", "Fior", "Burati", "Terlago",
    "Badelski",
    "Sardagna", "Francisci",
    "(Vedova)"
    "Angeli", "Parolin", "Bona", "Ferrari"
    "Emer", "Bevilaqua", "Bombardi", "Slop",
    "Piccola", 'Bedolski', "Rovetti", "Zoret",
    "Anderle", "Menghin", "Slop", "Slopp",
    "Rungg", "Bassetti", "Fedrigotti", "Maffei",
    "Bassetti", "Vescovi", "Petri", "Riolfatti",
    "(Schalzer)", "Schalzer", "Terlago"
]

In [277]:
prefissi = [
    "Don", "Conte",
    "V.Va", "Vedª", "Padre", "Conte", "Don",
    "(Vedova)","Vedova"
]


In [278]:
for index, row in passaporti.iterrows():
    if (row['nome'].find(" ") > -1):
        for cognome in cognomi_doppi:
            if row['nome'].find(cognome) > -1:
                v = row['nome']
                v = v.split(' ')
                new = row['cognome'] + " " +  v[0]
                passaporti.loc[index, 'cognome'] = new
                passaporti.loc[index, 'nome'] = row['nome'].replace(v[0], "").lstrip()


In [279]:
passaporti['eta_rilascio'] = passaporti['data'] - passaporti['data_nascita']

In [280]:
passaporti['mestiere'] = passaporti['mestiere'].str.lower()

In [281]:
del passaporti["numero_busta"]

In [282]:
luoghi = {
    'S. Orsola': "Sant'Orsola",
    "Baselga Di Pinè":  "Baselga di Pinè",
    "Ville Di Giovo": "Ville di Giovo",
    "Mezzotedesco" : "Mezzocorona",
    "Matterello" : "Mattarello",
    "Bedollo Pinè": "Bedollo",
    "Miola Di Pinè": "Miola",
    "Baselga Di Vezzano": "Baselga di Vezzano",
    "Baselga Di Pino":"Basegla di Pinè",
    "S. Michele":"San Michele",
    "Vigo Di Pinè":"Vigo di Pinè",
    "S. Rocco":"San Rocco",
    "Fierozzo S. Felice":"Fierozzo",
    "Miola Pinè": "Miola",
    "S.Massenza":"Santa Massenza",
    "Massenza": "Santa Massenza",
    "Bedol":"Bedollo",
    "Palù Di Giovo":"Palù di Giovo",
    "Masi Di Vigo": "Masi di Vigo",
    "Pergine": "Pergine Valsugana"
}


In [283]:
for index, row in passaporti.iterrows():
    luogo_nascita = row['luogo_nascita']
    for wrong_luogo in luoghi.keys():
        if (luogo_nascita == wrong_luogo): 
            passaporti.loc[index, 'luogo_nascita'] = luoghi[wrong_luogo]
            break


In [284]:
passaporti.shape

(12098, 18)

In [285]:
passaporti.drop_duplicates(inplace=True)


In [286]:
passaporti.shape


(12039, 18)